**Reading of the dataset**

In [1]:
import pandas as pd
import numpy as np
# csvs/hog_95/extracted_features_hog_95.csv
df1 = pd.read_csv('/home/swaksh/VSCode/prml_project/Face_Recognition/csvs/hog_95/extracted_features_hog_95.csv',header=None)
df2 = pd.read_csv('/home/swaksh/VSCode/prml_project/Face_Recognition/csvs/hog_95/extracted_features_test_hog_95.csv',header=None)

df = pd.concat([df1,df2])
print(df.shape)


(13233, 3249)


**Visuelise The occurence Of Each Name No. Of Times In the Dataset** 

In [2]:
name_counts = df.iloc[:,0].value_counts()

print(name_counts)

for value in df.iloc[:,0].value_counts():
    print(value)

0
George_W_Bush        530
Colin_Powell         236
Tony_Blair           144
Donald_Rumsfeld      121
Gerhard_Schroeder    109
                    ... 
Brian_De_Palma         1
David_Rivkin_Jr        1
Koichiro_Matsuura      1
Olivera_Labus          1
Jalen_Rose             1
Name: count, Length: 5749, dtype: int64
530
236
144
121
109
77
71
60
55
53
52
52
49
48
44
42
42
41
41
39
39
37
36
35
33
33
33
32
32
32
31
31
30
30
29
29
28
28
27
26
26
25
24
24
24
23
23
23
22
22
22
22
22
21
21
21
21
20
20
20
20
20
19
19
19
19
19
19
19
18
18
18
18
18
17
17
17
17
17
17
17
17
16
16
16
15
15
15
15
15
15
15
15
15
15
15
14
14
14
14
14
14
14
14
14
14
13
13
13
13
13
13
13
13
13
13
13
12
12
12
12
12
12
12
12
12
12
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
6
6
6
6


**Printing What Will Be The Size OF The Dataset**

In [3]:
count = 0
for value in name_counts:
    if(value >=10): count+= value
    
print(count)

4324


**Filtering The Dataset With Names That Occurs More than 10 Times** 

In [4]:
filtered_names_indices = name_counts[name_counts >= 10].index.to_list()

new_df_gte_10 = df[df.iloc[:,0].isin(filtered_names_indices)]

print(new_df_gte_10.shape)

(4324, 3249)


**Splitting Filtered Dataset Into Train and Test**

In [5]:
y = new_df_gte_10.iloc[:,0]
X = new_df_gte_10.drop(columns=[new_df_gte_10.columns[0]])
print(X.shape," ",y.shape)

(4324, 3248)   (4324,)


**Normalization Of The Dataset**

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)


In [7]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42)
print(X_train.shape,"\n",X_test.shape,"\n",y_train.shape,"\n",y_test.shape)

(3459, 3248) 
 (865, 3248) 
 (3459,) 
 (865,)


In [8]:
import numpy as np

unique_counts = np.unique(y_train, return_counts=True)

labels, counts = unique_counts

indices_greater_than_45 = np.where(counts >= 45)[0]
indices_less_than_45 = np.where(counts < 45)[0]

X_train_greater_than_45 = X_train[np.isin(y_train, labels[indices_greater_than_45])]
X_train_less_than_45 = X_train[np.isin(y_train, labels[indices_less_than_45])]

y_train_greater_than_45 = y_train[np.isin(y_train, labels[indices_greater_than_45])]
y_train_less_than_45 = y_train[np.isin(y_train, labels[indices_less_than_45])]

# Print the shapes of the resulting datasets
print("X_train >= 45:", X_train_greater_than_45.shape)
print("y_train >= 45:", y_train_greater_than_45.shape)
print("X_train < 45:", X_train_less_than_45.shape)
print("y_train < 45:", y_train_less_than_45.shape)


X_train >= 45: (1092, 3248)
y_train >= 45: (1092,)
X_train < 45: (2367, 3248)
y_train < 45: (2367,)


**Convert y_train and y_test into Numeric Values As ANN Trains On Numerical Data**

In [9]:
y_train_greater_than_45 = y_train_greater_than_45.values.flatten()
labels_train_gt45 ,unique_train_gt45 = pd.factorize(y_train_greater_than_45)

print(labels_train_gt45,"\n",len(unique_train_gt45))


[0 1 0 ... 0 5 0] 
 8


**Filtered Data Labels Calculate Value Count**

In [10]:
labels_count_train = pd.Series(labels_train_gt45).value_counts()
print(labels_count_train)

0    426
2    188
1    120
6     97
5     88
7     64
4     58
3     51
Name: count, dtype: int64


**Mapping The Names With The Numeric Value(Labels) Passed As The y_train To The Model** 

In [11]:
y_train_greater_than_45 = np.array(labels_train_gt45)
print(y_train_greater_than_45.shape)

(1092,)


In [12]:
labels_unique_gt45 = np.unique(labels_train_gt45)
print(labels_unique_gt45.shape, unique_train_gt45.shape)

(8,) (8,)


In [13]:
y_train_labels_to_names_gt45 = {}
for i in range(len(labels_unique_gt45)):
    y_train_labels_to_names_gt45[labels_unique_gt45[i]] = unique_train_gt45[i]
    
print(y_train_labels_to_names_gt45)
print(len(y_train_labels_to_names_gt45))

{0: 'George_W_Bush', 1: 'Tony_Blair', 2: 'Colin_Powell', 3: 'Junichiro_Koizumi', 4: 'Hugo_Chavez', 5: 'Gerhard_Schroeder', 6: 'Donald_Rumsfeld', 7: 'Ariel_Sharon'}
8


**Similarly For The Labels Count LessThan45**

In [15]:
y_train_less_than_45 = y_train_less_than_45.values.flatten()
labels_train_lt45 ,unique_train_lt45 = pd.factorize(y_train_less_than_45)

print(labels_train_lt45,"\n",len(unique_train_lt45))

labels_count_train = pd.Series(labels_train_lt45).value_counts()
print(labels_count_train)

y_train_less_than_45 = np.array(labels_train_lt45)
print(y_train_less_than_45.shape)

labels_unique_lt45 = np.unique(labels_train_lt45)
print(labels_unique_lt45.shape, unique_train_lt45.shape)

y_train_labels_to_names_lt45 = {}
for i in range(len(labels_unique_lt45)):
    y_train_labels_to_names_lt45[labels_unique_lt45[i]] = unique_train_lt45[i]
    
print(y_train_labels_to_names_lt45)
print(len(y_train_labels_to_names_lt45))

[  0   1   2 ... 125  18 114] 
 150
19     42
18     42
81     41
48     41
82     40
       ..
138     7
76      7
126     7
95      6
141     6
Name: count, Length: 150, dtype: int64
(2367,)
(150,) (150,)
{0: 'Tom_Hanks', 1: 'Silvio_Berlusconi', 2: 'Tom_Daschle', 3: 'John_Snow', 4: 'Renee_Zellweger', 5: 'John_Paul_II', 6: 'Hu_Jintao', 7: 'David_Beckham', 8: 'Jose_Maria_Aznar', 9: 'Hamid_Karzai', 10: 'Bill_Gates', 11: 'John_Bolton', 12: 'Atal_Bihari_Vajpayee', 13: 'Paul_Bremer', 14: 'Jennifer_Capriati', 15: 'Mahmoud_Abbas', 16: 'Igor_Ivanov', 17: 'Juan_Carlos_Ferrero', 18: 'Serena_Williams', 19: 'Jean_Chretien', 20: 'Dick_Cheney', 21: 'Norah_Jones', 22: 'Michael_Bloomberg', 23: 'Arnold_Schwarzenegger', 24: 'Lleyton_Hewitt', 25: 'Megawati_Sukarnoputri', 26: 'Alejandro_Toledo', 27: 'Julie_Gerberding', 28: 'Queen_Elizabeth_II', 29: 'Roh_Moo-hyun', 30: 'Naomi_Watts', 31: 'Charles_Moose', 32: 'Kim_Clijsters', 33: 'Alvaro_Uribe', 34: 'Rubens_Barrichello', 35: 'Lance_Armstrong', 36: 'Andre_A

**Train The Data On ANN Model And Predict**

In [24]:
import tensorflow as tf
import keras

model = keras.Sequential([
    keras.layers.Dense(2048,activation='relu',input_shape=(3248,)),
    keras.layers.Dense(1024,activation='relu'),
    keras.layers.Dense(512,activation='relu'),
    keras.layers.Dense(256,activation='relu'),
    keras.layers.Dense(150,activation='softmax'),  # Here 4883 are my unique labels
    
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


# X_train_less_than_45 = X_train_less_than_45.astype('float32')  # Convert to float if not already
# y_train_less_than_45 = y_train_less_than_45.astype('int32')  # Convert to int if not already

X_train_less_than_45 = np.array(X_train_less_than_45)
y_train_less_than_45 = np.array(y_train_less_than_45)

model.fit(X_train_less_than_45,y_train_less_than_45,epochs=300,batch_size=64)

X_test = np.array(X_test)

predictions_lt45 = model.predict(X_test)

Epoch 1/300


/home/swaksh/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - accuracy: 0.0350 - loss: 4.9606
Epoch 2/300
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - accuracy: 0.3604 - loss: 2.8227
Epoch 3/300
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - accuracy: 0.9094 - loss: 0.4377
Epoch 4/300
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - accuracy: 0.9862 - loss: 0.0797
Epoch 5/300
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - accuracy: 0.9815 - loss: 0.1108
Epoch 6/300
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - accuracy: 0.9889 - loss: 0.0480
Epoch 7/300
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - accuracy: 0.9977 - loss: 0.0158
Epoch 8/300
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - accuracy: 0.9986 - loss: 0.0097
Epoch 9/300
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - accuracy: 1.0000 - loss: 0.0019
Epoch 10/300
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - accuracy: 1.0000 - loss: 5.3945e-04
Epoch 11/300
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - accuracy: 1.0000 - loss: 2.5364e-04
Epoch 12/300
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - a

In [25]:
import tensorflow as tf
import keras

model = keras.Sequential([
    keras.layers.Dense(2048,activation='relu',input_shape=(3248,)),
    keras.layers.Dense(1024,activation='relu'),
    keras.layers.Dense(512,activation='relu'),
    keras.layers.Dense(256,activation='relu'),
    keras.layers.Dense(8,activation='softmax'),  # Here 4883 are my unique labels
    
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

print(y_train.shape)

X_train_greater_than_45 = np.array(X_train_greater_than_45)
y_train_greater_than_45 = np.array(y_train_greater_than_45)
model.fit(X_train_greater_than_45,y_train_greater_than_45,epochs=300,batch_size=64)


X_test = np.array(X_test)
y_test = np.array(y_test)
predictions_gt45 = model.predict(X_test)

/home/swaksh/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


(3459,)
Epoch 1/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - accuracy: 0.3725 - loss: 2.2224
Epoch 2/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.8607 - loss: 0.4543
Epoch 3/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.9855 - loss: 0.0406
Epoch 4/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.9962 - loss: 0.0157
Epoch 5/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.9976 - loss: 0.0075
Epoch 6/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.9995 - loss: 0.0074
Epoch 7/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.9960 - loss: 0.0059
Epoch 8/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 1.0000 - loss: 5.2471e-04
Epoch 9/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 1.0000 - loss: 4.4755e-04
Epoch 10/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 1.0000 - loss: 6.3729e-05
Epoch 11/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 1.0000 - loss: 6.0959e-05
Epoch 12/300
18/18 ━━━━━━━━━

In [37]:
predictions_gt45 = np.array(predictions_gt45)
predictions_lt45  = np.array(predictions_lt45)
print(predictions_gt45.shape)
print(predictions_lt45.shape)

(865, 8)
(865, 150)


In [44]:

y_pred_lt45= []
y_prob_lt45 = []

for prediction in predictions_lt45:
    y_pred_lt45.append(np.argmax(prediction)) 
    y_prob_lt45.append(prediction[np.argmax(prediction)])
print(len(y_pred_lt45))
y_pred_lt45 = np.array(y_pred_lt45)
y_prob_l45 = np.array(y_prob_lt45)

y_pred_names_lt45 = []
for label in y_pred_lt45:
    y_pred_names_lt45.append(y_train_labels_to_names_lt45[label])

print(len(y_pred_names_lt45))

865
865


In [45]:

y_pred_gt45= []
y_prob_gt45 = []

for prediction in predictions_gt45:
    y_pred_gt45.append(np.argmax(prediction)) 
    y_prob_gt45.append(prediction[np.argmax(prediction)])
print(len(y_pred_gt45))
y_pred_gt45 = np.array(y_pred_gt45)
y_prob_g45 = np.array(y_prob_gt45)

y_pred_names_gt45 = []
for label in y_pred_gt45:
    y_pred_names_gt45.append(y_train_labels_to_names_gt45[label])

print(len(y_pred_names_gt45))

865
865


In [49]:
count = 0
for i in range(len(y_pred_names_lt45)):
    # if y_prob_gt45 > y_prob_lt45:
    pred_name1 = y_pred_names_gt45[i]
    # else:
    pred_name2 = y_pred_names_lt45[i]
        
    if pred_name1 == y_test[i] or pred_name2 == y_test[i]:  
        print("y_pred_names is: ",pred_name1," or ",pred_name2," and ","y_test name is: ",y_test[i])
        count+=1

accuracy = count/865
print("\n\nAccuracy is this: ",accuracy)

y_pred_names is:  Tony_Blair  or  Jean_Chretien  and  y_test name is:  Jean_Chretien
y_pred_names is:  Tony_Blair  or  Jose_Maria_Aznar  and  y_test name is:  Tony_Blair
y_pred_names is:  George_W_Bush  or  Hans_Blix  and  y_test name is:  Hans_Blix
y_pred_names is:  George_W_Bush  or  Mohammad_Khatami  and  y_test name is:  Mohammad_Khatami
y_pred_names is:  George_W_Bush  or  George_Robertson  and  y_test name is:  George_W_Bush
y_pred_names is:  Colin_Powell  or  Paul_Bremer  and  y_test name is:  Colin_Powell
y_pred_names is:  Colin_Powell  or  John_Ashcroft  and  y_test name is:  Colin_Powell
y_pred_names is:  George_W_Bush  or  Eduardo_Duhalde  and  y_test name is:  George_W_Bush
y_pred_names is:  Colin_Powell  or  Hans_Blix  and  y_test name is:  Colin_Powell
y_pred_names is:  Hugo_Chavez  or  Serena_Williams  and  y_test name is:  Serena_Williams
y_pred_names is:  George_W_Bush  or  John_Kerry  and  y_test name is:  George_W_Bush
y_pred_names is:  Colin_Powell  or  Nicanor_Duar

**Mapping Back The Predicted Values To The Names**

In [53]:
y_pred_10 = []
for prediction in predictions_10:
    y_pred_10.append(np.argmax(prediction))
    
y_pred_10 = np.array(y_pred_10)
print(y_pred_10)

[  5  90  29 110  98  35  64   4   2   5 106  20  41  73  38  34  75   5
  20  77  62 117  65  22 101  93  20 135  70  55  78  85  98  46   5  31
  55  20   5 125  72  21  20  46  27   5  20  35  30   4  20  51  74 131
  55  86  74   5  62  85  14  41  16   8  40   5 101  48  98 155   5 107
   5   5  35  40  28   1  11 117  56 156  71  20  77  38   8   5 149 155
  54 121   5 106  77   1 121 155  17  89 141  98  83  25  65  54  17  60
 130  40  72  77  17  58  46   5  22  62  20   9  88  46  20  77  89 123
   1  92   5  74   8  28  94  86  43  20  54  63  20 107  35  88 131 119
 147  74 151  82  77  20  20  20  28  24 117 117 149  35  77   0  24  17
  36  74   1  35  48  25  35  27  28 145  43   5 105  46  60 147  90  46
  55 117 108  67   5 107  21 110  38  16   8  46 117  98   5  25   5  13
  38   8 132   8  33  46  21 122   0 113  88   5  41  89   8  79 121  35
  66 148  34  29 106  55 106   6   5  20   5 110   8  35  66   5  94  62
  22  20  74  20  90   1 140  27   5   8   5  97  4

In [54]:
y_pred_names_10 = []

for label in y_pred_10:
    y_pred_names_10.append(y_train_labels_to_names[label])
    
y_pred_names_10 = np.array(y_pred_names_10)
print(y_pred_names_10)

['George_W_Bush' 'Luiz_Inacio_Lula_da_Silva' 'Alejandro_Toledo'
 'Angelina_Jolie' 'John_Ashcroft' 'Hugo_Chavez' 'Jack_Straw'
 'Renee_Zellweger' 'Tom_Daschle' 'George_W_Bush' 'Nestor_Kirchner'
 'Colin_Powell' 'Andre_Agassi' 'Wen_Jiabao' 'Alvaro_Uribe'
 'Junichiro_Koizumi' 'Lindsay_Davenport' 'George_W_Bush' 'Colin_Powell'
 'Kofi_Annan' 'Winona_Ryder' 'Laura_Bush' 'John_Kerry' 'Jean_Chretien'
 'Trent_Lott' 'Amelie_Mauresmo' 'Colin_Powell' 'Mike_Weir' 'Jiri_Novak'
 'Donald_Rumsfeld' 'Jean_Charest' 'Bill_Simon' 'John_Ashcroft'
 'Gerhard_Schroeder' 'George_W_Bush' 'Queen_Elizabeth_II'
 'Donald_Rumsfeld' 'Colin_Powell' 'George_W_Bush' 'Lucio_Gutierrez'
 'Salma_Hayek' 'Serena_Williams' 'Colin_Powell' 'Gerhard_Schroeder'
 'Lleyton_Hewitt' 'George_W_Bush' 'Colin_Powell' 'Hugo_Chavez'
 'Julie_Gerberding' 'Renee_Zellweger' 'Colin_Powell' 'Tang_Jiaxuan'
 'Ariel_Sharon' 'Jackie_Chan' 'Donald_Rumsfeld' 'Spencer_Abraham'
 'Ariel_Sharon' 'George_W_Bush' 'Winona_Ryder' 'Bill_Simon'
 'Atal_Bihari_Vajpay

**Accuracy Calculation**

In [56]:
count = 0

for i in range(len(y_pred_names_10)):
    if y_pred_names_10[i] == y_test[i]:
        print("y_pred_names is: ",y_pred_names_10[i] ," and ","y_test name is: ",y_test[i])
        count+=1
        
accuracy = count/len(y_pred_names_10)
    
print("\n\nAccuracy is this: ",accuracy)

y_pred_names is:  Renee_Zellweger  and  y_test name is:  Renee_Zellweger
y_pred_names is:  Colin_Powell  and  y_test name is:  Colin_Powell
y_pred_names is:  George_W_Bush  and  y_test name is:  George_W_Bush
y_pred_names is:  Colin_Powell  and  y_test name is:  Colin_Powell
y_pred_names is:  George_W_Bush  and  y_test name is:  George_W_Bush
y_pred_names is:  Donald_Rumsfeld  and  y_test name is:  Donald_Rumsfeld
y_pred_names is:  George_W_Bush  and  y_test name is:  George_W_Bush
y_pred_names is:  Gerhard_Schroeder  and  y_test name is:  Gerhard_Schroeder
y_pred_names is:  Julie_Gerberding  and  y_test name is:  Julie_Gerberding
y_pred_names is:  George_W_Bush  and  y_test name is:  George_W_Bush
y_pred_names is:  George_W_Bush  and  y_test name is:  George_W_Bush
y_pred_names is:  John_Allen_Muhammad  and  y_test name is:  John_Allen_Muhammad
y_pred_names is:  George_W_Bush  and  y_test name is:  George_W_Bush
y_pred_names is:  George_W_Bush  and  y_test name is:  George_W_Bush
y_pr

**Seperately Training The HoG,CNN and lbp**

In [23]:
X_train = np.array(X_train)
X_test = np.array(X_test)

X_train_hog = X_train[:,1:945]
X_test_hog = X_test[:,1:945]

X_train_cnn = X_train[:,945:2993]
X_test_cnn = X_test[:,945:2993]

X_train_lbp = X_train[:,2993:]
X_test_lbp = X_test[:,2993:]

print(X_train_hog.shape,X_test_hog.shape,X_train_cnn.shape,X_test_cnn.shape,X_train_lbp.shape,X_test_lbp.shape)



(3459, 944) (865, 944) (3459, 2048) (865, 2048) (3459, 255) (865, 255)


**Train On HoG Features**

In [33]:
import tensorflow as tf
import keras
from keras import layers,models
import torch

model_hog = keras.Sequential([
    keras.layers.Dense(2048,activation='relu',input_shape=(944,)),
    # keras.layers.Dropout(0.5),
    keras.layers.Dense(1024,activation='relu'),
    # keras.layers.Dropout(0.5),
    keras.layers.Dense(512,activation='relu'),
    # keras.layers.Dropout(0.5),
    keras.layers.Dense(256,activation='relu'),
    # keras.layers.Dropout(0.5),
    keras.layers.Dense(158,activation='softmax'),  # Here 4883 are my unique labels
    
])

model_hog.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

print(y_train.shape)

X_train_hog = np.array(X_train_hog)
y_train = np.array(y_train)
print(X_train_hog.dtype,y_train.dtype)
model_hog.fit(X_train_hog,y_train,epochs=250,batch_size=64)


X_test_hog = np.array(X_test_hog)
y_test = np.array(y_test)
predictions_10_hog = model_hog.predict(X_test_hog)

(3459,)
float64 int64
Epoch 1/250


/home/swaksh/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.1308 - loss: 4.6380
Epoch 2/250
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.2615 - loss: 3.6041
Epoch 3/250
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.4784 - loss: 2.2389
Epoch 4/250
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.7717 - loss: 0.9340
Epoch 5/250
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.9411 - loss: 0.2501
Epoch 6/250
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.9912 - loss: 0.0516
Epoch 7/250
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.9988 - loss: 0.0099
Epoch 8/250
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.9992 - loss: 0.0062
Epoch 9/250
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.9983 - loss: 0.0082
Epoch 10/250
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 1.0000 - loss: 0.0015
Epoch 11/250
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 1.0000 - loss: 6.7334e-04
Epoch 12/250
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - accur

**From The Mapped lables To Names Get y_pred_hog Names And Accuracy Calculation**

In [ ]:
y_pred_hog = []
for pred in predictions_10_hog:
    y_pred_hog.append(np.argmax(pred))
    
y_pred_hog = np.array(y_pred_hog)
print(y_pred_hog)

In [ ]:
y_pred_hog_names = []

for label in y_pred_hog:
    y_pred_hog_names.append(y_train_labels_to_names[label])
    
y_pred_hog_names = np.array(y_pred_hog_names)
print(y_pred_hog_names)

count = 0
for i in range(len(y_pred_hog_names)):
    if y_pred_hog_names[i] == y_test[i]:
        print("y_pred_names of hog is: ",y_pred_hog_names[i] ," and ","y_test name is: ",y_test[i])
        count+=1
        
accuracy_hog = count/len(y_pred_hog_names)
    
print("\n\nAccuracy is this: ",accuracy_hog)

**Train On CNN Features**

In [ ]:
import tensorflow as tf
import keras
from keras import layers,models
import torch

model_cnn = keras.Sequential([
    keras.layers.Dense(2048,activation='relu',input_shape=(2048,)),
    # keras.layers.Dropout(0.5),
    keras.layers.Dense(1024,activation='relu'),
    # keras.layers.Dropout(0.5),
    keras.layers.Dense(512,activation='relu'),
    # keras.layers.Dropout(0.5),
    keras.layers.Dense(256,activation='relu'),
    # keras.layers.Dropout(0.5),
    keras.layers.Dense(158,activation='softmax'),  # Here 4883 are my unique labels
    
])

model_cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

print(y_train.shape)

X_train_cnn = np.array(X_train_cnn)
y_train = np.array(y_train)
model_cnn.fit(X_train_cnn,y_train,epochs=250,batch_size=64)

X_test_cnn = np.array(X_test_cnn)
y_test = np.array(y_test)
predictions_10_cnn = model_cnn.predict(X_test_cnn)

**From The Mapped lables To Names Get y_pred_cnn Names And Accuracy Calculation**

In [ ]:
y_pred_cnn = []
for prediction in predictions_10_cnn:
    y_pred_cnn.append(np.argmax(prediction))
    
y_pred_cnn = np.array(y_pred_cnn)
print(y_pred_cnn)

In [ ]:
y_pred_cnn_names = []

for label in y_pred_cnn:
    y_pred_cnn_names.append(y_train_labels_to_names[label])
    
y_pred_cnn_names = np.array(y_pred_cnn_names)
print(y_pred_cnn_names)

count = 0
for i in range(len(y_pred_cnn_names)):
    if y_pred_cnn_names[i] == y_test[i]:
        print("y_pred_names of cnn is: ",y_pred_cnn_names[i] ," and ","y_test name is: ",y_test[i])
        count+=1
        
accuracy_cnn = count/len(y_pred_cnn_names)
    
print("\n\nAccuracy is this: ",accuracy_cnn)

**Train Model On lbp Features**

In [ ]:
import tensorflow as tf
import keras
from keras import layers,models
import torch

model_lbp = keras.Sequential([
    keras.layers.Dense(2048,activation='relu',input_shape=(255,)),
    # keras.layers.Dropout(0.5),
    keras.layers.Dense(1024,activation='relu'),
    # keras.layers.Dropout(0.5),
    keras.layers.Dense(512,activation='relu'),
    # keras.layers.Dropout(0.5),
    keras.layers.Dense(256,activation='relu'),
    # keras.layers.Dropout(0.5),
    keras.layers.Dense(158,activation='softmax'),  # Here 4883 are my unique labels
    
])

model_lbp.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

print(y_train.shape)

X_train_lbp = np.array(X_train_lbp)
y_train = np.array(y_train)
model_lbp.fit(X_train_lbp,y_train,epochs=250,batch_size=64)

X_test_lbp = np.array(X_test_lbp)
y_test = np.array(y_test)
predictions_10_lbp = model_lbp.predict(X_test_lbp)

**From The Mapped lables To Names Get y_pred_lbp Names And Accuracy Calculation**

In [ ]:
y_pred_lbp = []
for prediction in predictions_10_lbp:
    y_pred_lbp.append(np.argmax(prediction))
    
y_pred_lbp = np.array(y_pred_lbp)
print(y_pred_lbp)

In [ ]:
y_pred_lbp_names = []

for label in y_pred_lbp:
    y_pred_lbp_names.append(y_train_labels_to_names[label])
    
y_pred_lbp_names = np.array(y_pred_lbp_names)
print(y_pred_lbp_names)

count = 0
for i in range(len(y_pred_lbp_names)):
    if y_pred_lbp_names[i] == y_test[i]:
        print("y_pred_names of lbp is: ",y_pred_lbp_names[i] ," and ","y_test name is: ",y_test[i])
        count+=1
        
accuracy_lbp = count/len(y_pred_lbp_names)
    
print("\n\nAccuracy is this: ",accuracy_lbp)